In [22]:
%%html
<style>
body,h1,h2,h3,h4,h5,p,div{font-family:'ubuntu' !important};
.CodeMirror-line{font-family:'Fira Sans Book' !important}
</style>


<h1>Kubernetes</h1>



<img src="assets/k8s-logo.png" alt="k8s-logo" style="width: 400px;"/>

### Agenda
* Was ist Kubernetes?
* Grundlagen
* kubectl
* Deployment Beispiel

# Was ist Kubernetes


* Programm zum automatisierten Deployment

* basierend auf der Container Runtime von Docker

* cloud-native

# verteilt

* Kubernetes verteilt Deployments über verschiedene Rechner
* eine Instanz heißt Cluster und besteht aus mehreren Nodes

# zuverlässig und hochverfügbar
* Kubernetes sorgt selbst dafür, dass Deployments laufen (self-healing)
* Deployments erfolgen ohne Downtime

# skalierbar
* Deployments können je nach Last automatisch skaliert werden
* Clusterresourcen können dynamisch erweitert werden (Nodes hinzufügen)

# Kubernetes und Microservices
K8s begünstigt Micorservice Architekturen  
-> und somit agile Arbeitsweisen 

# Grundlagen

<h1> Deklarative Konfiguration  <br>oder Infrastructure as code</h1>





* alles in K8s sind API-Resourcen
* Interaktionen mit Kubernetes erfolgen über diese API (kubectl)
* Befehle können ausgeführt werden

* **besser:** keine Anweisungen ausführen, sondern nur Config-Dateien bereitstellen
* Verwaltung der Config-Dateien mit VCS
* Ziel: Zustand des Clusters entspricht dem in der Versionsverwaltung

# ein paar Kubernetes Resourcen


<img src="assets/ressources.png" alt="k8s-logo" style="max-height:90vh"/>

# Pod
* enthält einen mehrer Container
* Conatainer teilen sich Netzwerk und eventuell Speicher

In [17]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   praesi.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/

no changes added to commit (use "git add" and/or "git commit -a")
